In [33]:
import jax.numpy as jnp
import numpy as np
import os as os
import pylab as plt
import jax
import time
import pandas as pd
from itertools import product
from scipy.linalg import fractional_matrix_power

## Optimizing the Rosenbrock function with different optimizers 

### Rosenbrock function 
$$
F(x_1, x_2) = 100 \times (x_2 - x_1^2)^2 + (1-x_1)^2 
$$

We know that (1,1) is the global minimum with minimum value 0. We will try to minimize this function with an initial guess (0,0)

In [3]:
@jax.jit
def rosenbrock(x): 
    x1, x2 = x
    return 100 * (x2 - x1 ** 2) ** 2 + (1 - x1) ** 2

value_and_grad_rosenbrock = jax.jit(jax.value_and_grad(rosenbrock))

x_init = np.array([0., 0.])

We will minimize the function using 
1. Naive gradient descent 
2. Gradient descent with ADAM optimizer 
3. Gradient descent with RMSprop optimizer
4. Gradient descent with Shampoo optimizer

### 1. Naive Gradient descent 

In [29]:
learning_rates = np.logspace(-6, -1, num = 20)  # Learning rates from 1e-6 to 1
num_iterations = 1000

results = pd.DataFrame(columns=['Learning Rate', 'Final Loss', 'Loss Variance', 'Time Taken (s)'])

for lr in learning_rates:
    x = np.copy(x_init)
    losses = []
    
    start_time = time.time()

    for _ in range(num_iterations):
        current_loss, grad = value_and_grad_rosenbrock(x)
        losses.append(current_loss)
        
        x -= lr * grad  # Gradient descent update
    
    end_time = time.time() 

    final_loss = losses[-1]
    loss_variance = np.var(losses)
    time_taken = end_time - start_time

    # Store results in a DataFrame
    results = pd.concat([results, pd.DataFrame({
        'Learning Rate': [lr],
        'Final Loss': [final_loss],
        'Loss Variance': [loss_variance],
        'Time Taken (s)': [time_taken]
    })], ignore_index=True)

results['Final Loss'] = results['Final Loss'].astype(float)
results = results.sort_values(by = 'Final Loss')
results.head()


,Learning Rate,Final Loss,Loss Variance,Time Taken (s)
12,0.001438,0.060175,0.046657,0.057846
11,0.000785,0.147262,0.048067,0.085769
10,0.000428,0.280596,0.039403,0.081781
9,0.000234,0.447978,0.024860,0.113697
8,0.000127,0.619794,0.012051,0.137633


### 2. ADAM Optimizer

In [18]:
# Range of values that we are trying 
learning_rates = np.logspace(-6, -1, num=20)  
beta1_vals = [0.9, 0.85, 0.95]
beta2_vals = [0.999, 0.99, 0.95]
max_iter = 1000

# Initialize results DataFrame
results = pd.DataFrame(columns=['Learning Rate', 'Beta1', 'Beta2', 'Final Loss', 'Loss Variance', 'Time Taken (s)'])

# Adam optimizer parameters
epsilon = 1e-8

# Perform grid search over learning rate and beta values
for lr, beta1, beta2 in product(learning_rates, beta1_vals, beta2_vals):
    x = np.copy(x_init)  # Initialize x
    loss_hist = []  # Store loss history for variance calculation

    # Initialize Adam-specific variables
    g = np.zeros(2)
    v = np.zeros(2)
    
    # Start timer
    start_time = time.time()

    # Adam optimization loop
    for it in range(max_iter):
        current_loss, grad = value_and_grad_rosenbrock(x)  # Calculate loss and gradient
        loss_hist.append(current_loss)

        # Update running averages for gradient and squared gradient
        g = beta1 * g + (1 - beta1) * grad
        v = beta2 * v + (1 - beta2) * (grad ** 2)

        # Bias correction
        g_corrected = g / (1 - beta1 ** (it + 1))
        v_corrected = v / (1 - beta2 ** (it + 1))

        # Compute normalized gradient for Adam update
        grad_normalized = g_corrected / (np.sqrt(v_corrected) + epsilon)

        # Update x
        x -= lr * grad_normalized

    # End timer
    end_time = time.time()
    
    # Calculate metrics for this combination of parameters
    final_loss = loss_hist[-1]
    loss_variance = np.var(loss_hist)
    time_taken = end_time - start_time

    # Store results in DataFrame
    results = pd.concat([results, pd.DataFrame({
        'Learning Rate': [lr],
        'Beta1': [beta1],
        'Beta2': [beta2],
        'Final Loss': [final_loss],
        'Loss Variance': [loss_variance],
        'Time Taken (s)': [time_taken]
    })], ignore_index=True)

results = results.sort_values(by='Final Loss')
results.head()


,Learning Rate,Beta1,Beta2,Final Loss,Loss Variance,Time Taken (s)
168,0.054556,0.95,0.999,0.0,0.007362,0.430824
141,0.008859,0.95,0.999,2.2737368e-13,0.022579,0.312198
135,0.008859,0.90,0.999,1.4352963e-12,0.022999,0.299195
132,0.004833,0.95,0.999,2.689049e-11,0.038032,0.314193
159,0.029764,0.95,0.999,5.685621e-10,0.008537,0.530584


### 3. RMSprop Optimizer

In [32]:
# gradient descent for RMSprop
# Gradient descent with RMSProp
learning_rates = np.logspace(-6, -1, num=5)  # Test learning rates from 1e-6 to 0.1
betas = [0.7, 0.8, 0.9, 0.99]  # Test a few values for beta

# Initialize a DataFrame to store the results
results = pd.DataFrame(columns=['Learning Rate', 'Beta', 'Final Loss', 'Loss Variance', 'Time Taken (s)'])

# Parameters
epsilon = 1e-8
max_iter = 1000

for lr, beta in product(learning_rates, betas):
    x = np.copy(x_init)
    losses = []
    v = np.zeros(2)  # Running average of squared gradient, initialized to 0

    # Start timer for this combination of hyperparameters
    start_time = time.time()

    # Run gradient descent with RMSProp
    for it in range(max_iter): 
        current_loss, grad = value_and_grad_rosenbrock(x)
        losses.append(current_loss)

        # Update the running average of the squared gradient
        v = (1 - beta) * v + beta * (grad ** 2)

        # Compute the normalized gradient
        grad_normalized = grad / (np.sqrt(v) + epsilon)

        # Update parameters
        x = x - lr * grad_normalized

    # End timer
    end_time = time.time()

    # Collect results
    final_loss = losses[-1]
    loss_variance = np.var(losses)
    time_taken = end_time - start_time

    # Append results to DataFrame
    results = pd.concat([results, pd.DataFrame({
        'Learning Rate': [lr],
        'Beta': [beta],
        'Final Loss': [final_loss],
        'Loss Variance': [loss_variance],
        'Time Taken (s)': [time_taken]
    })], ignore_index=True)

results = results.sort_values(by='Final Loss')
results.head()

,Learning Rate,Beta,Final Loss,Loss Variance,Time Taken (s)
12,0.005623,0.70,0.035552815,0.026944,0.269815
13,0.005623,0.80,0.04610299,0.025752,0.252068
14,0.005623,0.90,0.073455006,0.023495,0.256473
15,0.005623,0.99,0.22157194,0.013768,0.246529
8,0.000316,0.70,0.46794412,0.023678,0.269163


### 4. Shampoo Optimizer

In [52]:
learning_rates = np.logspace(-6, 0, num = 10)  # Learning rates from 1e-6 to 0.1
max_iter = 1000

# Initialize a DataFrame to store the results
results = pd.DataFrame(columns=['Learning Rate', 'Final Loss', 'Loss Variance', 'Time Taken (s)'])  
epsilon = 1e-2
max_iter = 1000

for lr in learning_rates:
    loss_hist = []
    x = np.copy(x_init)

    # Initialize Shampoo-specific matrices
    L = np.zeros((2, 2))  
    R = np.zeros(1)  

    start_time = time.time()

    for it in range(max_iter):
        current_loss, grad = value_and_grad_rosenbrock(x)
        loss_hist.append(current_loss)

        grad = np.clip(grad, -1e3, 1e3)  # Adjust the clip range as needed
        
        # Update the gradient accumulation matrices
        L += np.outer(grad, grad) + epsilon * np.eye(2) # Outer product for Shampoo update
        R += np.dot(grad, grad) + epsilon # Inner product for Shampoo update
        
        # Compute the matrix root raised to the -1/4 power with epsilon for numerical stability
        L_root = fractional_matrix_power(L, 0.25)
        R_root = R ** (-1/4)

        # Use np.linalg.solve instead of np.linalg.inv for L_root and R_root
        L_inv_root = np.linalg.solve(L_root, np.eye(2))
        R_inv_root = 1 / R_root

        # Precondition the gradient with L^(-1/4) * grad * R^(-1/4)
        preconditioned_grad = L_inv_root @ grad * R_inv_root

        # Update parameters with the preconditioned gradient
        x = x - lr * preconditioned_grad

    end_time = time.time()

    # Collect results
    final_loss = loss_hist[-1]
    loss_variance = np.var(loss_hist)
    time_taken = end_time - start_time

    # Append results to DataFrame
    results = pd.concat([results, pd.DataFrame({
        'Learning Rate': [lr],
        'Final Loss': [final_loss],
        'Loss Variance': [loss_variance],
        'Time Taken (s)': [time_taken]
    })], ignore_index=True)

results = results.sort_values(by='Final Loss')
results.head()

,Learning Rate,Final Loss,Loss Variance,Time Taken (s)
5,0.002154,0.016199157,0.042131,0.528066
4,0.000464,0.22621474,0.045951,0.537434
3,0.000100,0.6761303,0.008754,0.537843
2,0.000022,0.9175868,0.000567,0.618315
1,0.000005,0.98162365,0.000028,0.883934
